<a href="https://colab.research.google.com/github/ali1810/Deep_Chem/blob/main/Solubility_Curated_datset_LR76%25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

In [8]:
sol = pd.read_csv('/content/drive/MyDrive/curated-solubility-dataset.csv')
sol

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,MolLogP,MolMR,HeavyAtomCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.000000,1,G1,392.510,3.95810,102.4454,23.0,0.0,0.0,2.0,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.000000,1,G1,169.183,2.40550,51.9012,13.0,1.0,1.0,2.0,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.000000,1,G1,140.569,2.15250,36.8395,9.0,1.0,0.0,2.0,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.000000,1,G1,756.226,8.11610,200.7106,53.0,6.0,2.0,7.0,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.000000,1,G1,422.525,2.48540,119.0760,31.0,6.0,0.0,6.0,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9977,I-84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000,0.000000,1,G1,264.369,2.61700,78.6762,19.0,4.0,1.0,4.0,8.0,106.0,1.0,0.0,0.0,1.0,41.57,115.300645,2.394548e+00,374.236893
9978,I-85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000,0.000000,1,G1,444.440,-0.21440,109.5409,32.0,9.0,6.0,10.0,2.0,170.0,1.0,0.0,3.0,4.0,181.62,182.429237,2.047922e+00,1148.584975
9979,I-86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000,0.019222,3,G5,150.221,2.82402,46.9328,11.0,1.0,1.0,1.0,1.0,60.0,1.0,0.0,0.0,1.0,20.23,67.685405,3.092720e+00,251.049732
9980,I-93,verapamil,"InChI=1S/C27H38N2O4/c1-20(2)27(19-28,22-10-12-...",SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,-3.980000,0.000000,1,G1,454.611,5.09308,131.6560,33.0,6.0,0.0,6.0,13.0,180.0,2.0,0.0,0.0,2.0,63.95,198.569223,2.023333e+00,938.203977


In [9]:
sol.SMILES

0                     [Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C
1                                    O=C1Nc2cccc3cccc1c23
2                                         Clc1ccc(C=O)cc1
3       [Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...
4       C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...
                              ...                        
9977                       C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C
9978    OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...
9979                                  c1(cc(ccc1C(C)C)C)O
9980    COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...
9981                CC(=O)CC(c1ccccc1)c1c(O)c2ccccc2oc1=O
Name: SMILES, Length: 9982, dtype: object

In [29]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:39
🔁 Restarting kernel...


In [1]:
### Installing RDKIT 
!mamba install -c conda-forge rdkit

Streaming output truncated to the last 5000 lines.
libcblas                 [] (00m:00s) Waiting...
xorg-xextproto           [] (00m:00s) Waiting...
freetype                 [] (00m:00s) Waiting...
rdkit                    [] (00m:00s)    943 KB /     38 MB (727.06 KB/s)
openssl                  [] (00m:01s) Waiting...
boost-cpp                [] (00m:00s) Waiting...
libpng                   [] (00m:00s) Decompressing...
libgfortran5             [] (00m:00s) Waiting...
libwebp-base             [] (00m:00s) Waiting...
olefile                  [] (00m:00s) Waiting...
pcre                     [] (00m:00s) Waiting...
pixman                   [] (00m:00s) Waiting...
pyparsing                [] (00m:00s) Waiting...
pytz                     [] (00m:00s) Waiting...
matplotlib-base          [] (00m:00s) Waiting...
xorg-kbproto             [] (00m:00s) Waiting...
tornado                  [] (00m:00s) Waiting...
xorg-libsm               [] (00m:00s) Waiting...
xorg-libxext             [] (00m:00s

**Convert SMILES in t molecule **

In [2]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

In [10]:
mol_list= []
for element in sol.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:10:47] WARNING: not r

In [ ]:
len(mol_list)

9982

Calculate molecular descriptors
To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:

cLogP (Octanol-water partition coefficient)
MW (Molecular weight)
RB (Number of rotatable bonds)
AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)
Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute. 

####  Calculate LogP, MW and RB

In [11]:
import numpy as np
from rdkit.Chem import Descriptors

In [12]:
### Write a function to calculate these values....

def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors



In [13]:
#### Get new dataframe 
df = generate(sol.SMILES)
df.head(100)

RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [09:11:06] WARNING: not r

,MolLogP,MolWt,NumRotatableBonds
0,3.9581,392.510,17.0
1,2.4055,169.183,0.0
2,2.1525,140.569,1.0
3,8.1161,756.226,10.0
4,2.4854,422.525,12.0
...,...,...,...
95,1.1989,176.256,7.0
96,2.2430,222.288,3.0
97,4.6019,385.637,19.0
98,4.2338,469.929,8.0


3.2.2. Aromatic proportion

3.2.1.1. Number of aromatic atoms

Here, we will create a custom function to calculate the Number of aromatic atoms. With this descriptor we can use it to subsequently calculate the AP descriptor.

Computing for a single molecule.

In [23]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')

NameError: ignored

In [ ]:
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

[False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True]

In [14]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [15]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
#desc_AromaticAtoms

###  Number of heavy atoms
Here, we will use an existing function for calculating the Number of heavy atoms.

In [16]:
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
#desc_HeavyAtomCount

###  Computing the Aromatic Proportion (AP) descriptor
Computing for a single molecule.

In [17]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
AromaticAtoms(m)/Descriptors.HeavyAtomCount(m)

0.5588235294117647

Computing for molecules in the entire dataset.



In [18]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
#desc_AromaticProportion

In [23]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
#df_desc_AromaticProportion

In [20]:
df

,MolLogP,MolWt,NumRotatableBonds
0,3.95810,392.510,17.0
1,2.40550,169.183,0.0
2,2.15250,140.569,1.0
3,8.11610,756.226,10.0
4,2.48540,422.525,12.0
...,...,...,...
9977,2.61700,264.369,8.0
9978,-0.21440,444.440,2.0
9979,2.82402,150.221,1.0
9980,5.09308,454.611,13.0


In [24]:
df_desc_AromaticProportion

,AromaticProportion
0,0.000000
1,0.769231
2,0.666667
3,0.679245
4,0.387097
...,...
9977,0.315789
9978,0.187500
9979,0.545455
9980,0.363636


In [41]:
### Comining in single dataframe ...  
### To make X matrix 
new_df=sol[["NumValenceElectrons",	"NumSaturatedRings",	"NumAliphaticRings",	"RingCount","NumHAcceptors",	"NumHDonors",	"NumHeteroatoms"]]
X = pd.concat([df,df_desc_AromaticProportion,new_df], axis=1)
X
#new_df

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion,NumValenceElectrons,NumSaturatedRings,NumAliphaticRings,RingCount,NumHAcceptors,NumHDonors,NumHeteroatoms
0,3.95810,392.510,17.0,0.000000,142.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2.40550,169.183,0.0,0.769231,62.0,0.0,1.0,3.0,1.0,1.0,2.0
2,2.15250,140.569,1.0,0.666667,46.0,0.0,0.0,1.0,1.0,0.0,2.0
3,8.11610,756.226,10.0,0.679245,264.0,0.0,0.0,6.0,6.0,2.0,7.0
4,2.48540,422.525,12.0,0.387097,164.0,4.0,4.0,6.0,6.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
9977,2.61700,264.369,8.0,0.315789,106.0,0.0,0.0,1.0,4.0,1.0,4.0
9978,-0.21440,444.440,2.0,0.187500,170.0,0.0,3.0,4.0,9.0,6.0,10.0
9979,2.82402,150.221,1.0,0.545455,60.0,0.0,0.0,1.0,1.0,1.0,1.0
9980,5.09308,454.611,13.0,0.363636,180.0,0.0,0.0,2.0,6.0,0.0,6.0


In [26]:
### To make Y matrix .....
sol.SMILES 

0                     [Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C
1                                    O=C1Nc2cccc3cccc1c23
2                                         Clc1ccc(C=O)cc1
3       [Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...
4       C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...
                              ...                        
9977                       C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C
9978    OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...
9979                                  c1(cc(ccc1C(C)C)C)O
9980    COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...
9981                CC(=O)CC(c1ccccc1)c1c(O)c2ccccc2oc1=O
Name: SMILES, Length: 9982, dtype: object

In [27]:
Y = sol.iloc[:,5]
Y

0      -3.616127
1      -3.254767
2      -2.177078
3      -3.924409
4      -4.662065
          ...   
9977   -3.010000
9978   -2.930000
9979   -2.190000
9980   -3.980000
9981   -4.780000
Name: Solubility, Length: 9982, dtype: float64

In [29]:
####  data split ...

from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, Y, test_size=0.2, random_state=42)

In [43]:
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)




LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:
model_svr=SVR(kernel='rbf')
model_svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [46]:
model_rf = RandomForestRegressor(n_estimators = 2000, random_state = 0)
model_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [35]:
print(y_test)

1253   -5.984242
9930   -4.160000
7046   -2.283900
8711   -4.230000
6880    0.552600
          ...   
2249   -2.483388
210    -3.396367
4961   -3.670200
1919   -0.391024
718    -1.753126
Name: Solubility, Length: 1997, dtype: float64


In [36]:
Y_pred_train = model.predict(X_train)



In [37]:
Y_pred_train_s = model_svr.predict(X_train)

In [38]:
Y_pred_test_svr = model_svr.predict(X_test)

In [47]:
Y_pred_test = model_rf.predict(X_test)
res = "\n".join("{} {}".format(x, y) for x, y in zip(Y_pred_test, y_test))
print(res)

-3.8910599262182117 -5.9842424969
-3.902652995315176 -4.16
-3.8390331899558765 -2.2839
-4.503433755286276 -4.23
-0.7181999641188777 0.5526
-2.092944713417195 -2.1531779235
-4.497097889135643 -4.7932278922
-2.50944142131552 -2.39
-1.5312225413933425 0.16653545109999998
-1.2417323567289307 -1.4969
-4.086768563941664 -4.581332510899999
-0.04906644202683282 1.0546594571
-1.0847036357893236 0.5917902317
-2.245699897193923 -1.89
-3.456104438822607 -5.9415611201999985
-3.8089298450590854 -3.79
0.45931729673081767 0.74
-5.88620535207959 -5.95
-2.1998609313093707 -2.92
-5.9605518879679185 -3.0376
-1.3440712473292031 -2.1
-3.3260673953652162 -2.9676237394
-2.5790075755493707 -1.9916
-0.9398649586505256 -2.4758
-3.9713621665860757 -4.63
-1.1509799609375009 -2.1520684654
-4.838823323048142 -7.208650928200001
-3.2001250971698063 -4.5171
-3.085995183723985 -3.1284
-4.743565032216179 -5.1563604082
-3.3736451982899744 -4.7886
0.04470214191566879 -0.9656
-5.8926703781606875 -6.8863
-1.0379416868880575 

In [50]:
from sklearn.metrics import r2_score
r2_score(y_test, Y_pred_test)

0.7631546650330788